In [ ]:
# Asthetics
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import json
import os
import random
from tqdm.autonotebook import tqdm
import string
import re
import glob
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

%matplotlib inline

os.listdir('/kaggle/input/coleridgeinitiative-show-us-the-data')

In [ ]:
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'
train_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/train/*.json")
test_files = glob.glob("../input/coleridgeinitiative-show-us-the-data/test/*.json")

In [ ]:
def basic_eda(df, row_limit=5, list_elements_limit=10):
    ### rows and columns
    print('Info : There are {} columns in the dataset'.format(df.shape[1]))
    print('Info : There are {} rows in the dataset'.format(df.shape[0]))
    
    print("==================================================")
    
    ## data types
    print("\nData type information of different columns")
    dtypes_df = pd.DataFrame(df.dtypes).reset_index().rename(columns={0:'dtype', 'index':'column_name'})
    cat_df = dtypes_df[dtypes_df['dtype']=='object']
    num_df = dtypes_df[dtypes_df['dtype']!='object']
    print('Info : There are {} categorical columns'.format(len(cat_df)))
    print('Info : There are {} numerical columns'.format(len(dtypes_df)-len(cat_df)))
    
    if list_elements_limit >= len(cat_df):
        print("Categorical columns : ", list(cat_df['column_name']))
    else:
        print("Categorical columns : ", list(cat_df['column_name'])[:list_elements_limit])
        
    if list_elements_limit >= len(num_df):
        print("Numerical columns : ", list(num_df['column_name']))
    else:
        print("Numerical columns : ", list(num_df['column_name'])[:list_elements_limit])
    
    #dtypes_df['dtype'].value_counts().plot.bar()
    display(dtypes_df.head(row_limit))
    
#     print("==================================================")
#     print("\nDescription of numerical variables")
    
#     #### Describibg numerical columns
#     desc_df_num = df[list(num_df['column_name'])].describe().T.reset_index().rename(columns={'index':'column_name'})
#     display(desc_df_num.head(row_limit))
    
    print("==================================================")
    print("\nDescription of categorical variables")
    
    desc_df_cat = df[list(cat_df['column_name'])].describe().T.reset_index().rename(columns={'index':'column_name'})
    display(desc_df_cat.head(row_limit))
    
    return

In [ ]:
train = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
train.head()

It looks like there are only 14,316 unique IDs in the dataset, meaning that some publications include a multitude of datasets.   
notice that the pub_title unique count is slightly smaller than the Id unique counts.   
This points to the precense of several occurences of having 2 separate publications, eahc with a unique ID, but sharing the exact same title.

Also, there are a total of 45 unique dataset_title and 130 unique dataset_label. Meaning that a single dataset could have multible labels throughout different publications.

## Multiple public titles with same Ids

In [ ]:
print("Printing below the public titles that have multiple Ids associated with them:\n")
public_titles_unique = train["pub_title"].unique()
for pub_title in public_titles_unique:
    if len(train[train["pub_title"] == pub_title]["Id"].unique()) > 1:
        print(f"'{pub_title}':", list(train[train["pub_title"] == pub_title]["Id"].unique()), "\n")

In [ ]:
basic_eda(train)

1) There are duplicate id's meaning that there are some pulications that are using mutiple datasets. That's why that id is repeating  
2) Same is the case with pub_title. A single publication is using mutiple datasets.  
3) There is NO one to one mapping of id and pub_title. Meaning that there are cases when two different publications (from two different authors) have same title. Well, interesting.      
4) There 45 dataset titles but 130 dataet labels. Meaning that there are some datasets that has multiple labels. We'll look into how these two are related.

## 1.1. Dataset Title VS Dataset Label 


In [ ]:
print("Printing below the dataset titles that have multiple dataset labels associated with them:\n")
datasets_titles_unique = train["dataset_title"].unique()
for dataset_title in datasets_titles_unique:
    if len(train[train["dataset_title"] == dataset_title]["dataset_label"].unique()) > 1:
        print(f"'{dataset_title}':", list(train[train["dataset_title"] == dataset_title]["dataset_label"].unique()), "\n")


In [ ]:
unique_titles = train['dataset_title'].unique()
dup_title = []
count = []
dup_list = []
for ut in unique_titles:
    title_df = train[train['dataset_title'] == ut]
    tdf = title_df[['Id', 'dataset_title', 'dataset_label']].drop_duplicates('dataset_label')
    if tdf.shape[0] > 1:
        #print(ut)
        dup_title.append(ut)
        count.append(tdf.shape[0])
        dup_list.append(list(tdf['dataset_label']))
        
dup_df = pd.DataFrame({'dataset_title':dup_title, 'label_count':count, 'label_list':dup_list})

In [ ]:
dup_df.set_index('dataset_title')['label_count'].sort_values(ascending=False).plot.barh(figsize=(12,18))
plt.title("No of labels that a dataset have")
plt.xlabel('labels_count')
plt.show()

## 1.2. Datasets Popularity

In [ ]:
dataset_titles_counts = train['dataset_title'].value_counts()
fig = go.Figure(data=[go.Table(
  columnwidth = [0.25, 2, 0.5],
  header=dict(
    values=["<b>Rank</b>", "<b>Dataset Title</b>", "<b>Mentions</b>"],
    line_color='darkslategray',
    fill_color="royalblue",
    align='center',
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=np.array([np.array((str(i+1), "<i>" + x + "</i>", "<b>" + str(y) + "</b>", )) for i, (x, y) in enumerate(zip(dataset_titles_counts.index, dataset_titles_counts.values))]).T,
    line_color='darkslategray',
    # 2-D list of colors for alternating rows
    fill_color = [["white","lavender"]*25],
    align = 'center',
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title={"text": "<b>Datasets Titles Mentions Counts</b>",
           "x": 0.5,
           "xanchor":"center",
           "font_size": 22},
    margin={"r":20, "l":20})

fig.show()

## 1.3. Datasets Occurence Together

In [ ]:
multi_mentions = [] # list of tuples (Id, num_unique_mentions) 

for Id in train.Id.unique():
    num_unique_mentions = len(train[train['Id'] == Id]['dataset_title'].unique())
    if num_unique_mentions > 1:
        multi_mentions.append((Id, num_unique_mentions))

print(f"There are {len(multi_mentions)} publications in the training set that mention more than 1 dataset. That is {len(multi_mentions) / len(train.Id.unique()) * 100:.2f}% of the publications.")

In [ ]:
co_mentions = defaultdict(int)

for Id, num_mentions in multi_mentions:
    co_mentions[tuple(sorted(train[train['Id'] == Id]["dataset_title"].unique()))] += 1

print(f"There are {len(co_mentions)} unique sets of co-occurences of dataset mentions.")

In [ ]:
co_mentions = dict(sorted(co_mentions.items(), key=lambda item: item[1], reverse=True))

fig = go.Figure(data=[go.Table(
  columnwidth = [0.25, 2, 0.5],
  header=dict(
    values=["<b>Rank</b>", "<b>Datasets Co-Mentions Sets</b>", "<b>Occurences</b>"],
    line_color='darkslategray',
    fill_color="royalblue",
    align='center',
    font=dict(color='white', size=12)
  ),
  cells=dict(
    values=np.array([np.array((str(i+1), "<i>" + str(k) + "</i>", "<b>" + str(v) + "</b>", )) for i, (k, v) in enumerate(co_mentions.items())]).T,
    line_color='darkslategray',
    fill_color = [["white","lavender"]*60],
    align = 'center',
    font = dict(color = 'darkslategray', size = 11)
    ))
])

fig.update_layout(
    title={"text": "<b>Datasets Mentioned in the same Publications</b>",
           "x": 0.5,
           "xanchor":"center",
           "font_size": 22},
    margin={"r":20, "l":20})

fig.show()

## Train Json

In [ ]:
print("Files in train directory : \n")
print(len(os.listdir('../input/coleridgeinitiative-show-us-the-data/train')))
print(os.listdir('../input/coleridgeinitiative-show-us-the-data/train')[:5])

In [ ]:
print("\nFiles in test directory : \n")
print(len(os.listdir('../input/coleridgeinitiative-show-us-the-data/test')))
print(os.listdir('../input/coleridgeinitiative-show-us-the-data/test')[:5])

In [ ]:
with open('../input/coleridgeinitiative-show-us-the-data/train/d0fa7568-7d8e-4db9-870f-f9c6f668c17b.json') as f:
    sample = json.load(f)
sample

In [ ]:
train[train.Id=='d0fa7568-7d8e-4db9-870f-f9c6f668c17b']

## Data Set preparation

In [ ]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
tqdm.pandas()
train['text'] = train['Id'].progress_apply(read_append_return)

In [ ]:
train.to_csv('train_with_text.csv')

In [ ]:
tqdm.pandas()
# Generate the training publications dataframe
df_train_publications = pd.DataFrame()

for train_file in train_files:
    file_data = pd.read_json(train_file)
    file_data.insert(0,'pub_id', train_file.split('/')[-1].split('.')[0])
    df_train_publications = pd.concat([df_train_publications, file_data])
df_train_publications.to_csv("df_train_publications.csv",index=False)

In [ ]:
tqdm.pandas()
# Generate the training publications dataframe
df_test_publications = pd.DataFrame()

for test_file in test_files:
    file_data = pd.read_json(train_file)
    file_data.insert(0,'pub_id', test_file.split('/')[-1].split('.')[0])
    df_test_publications = pd.concat([df_test_publications, file_data])
df_test_publications.to_csv("df_test_publications.csv",index=False)